In [ ]:
!pip install /kaggle/input/iterative-stratification/iterative-stratification-master/

In [ ]:
import sys
package_path = '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master'
sys.path.append(package_path)

In [ ]:
!ls ../input/efficientnet-pytorch/EfficientNet-PyTorch

In [ ]:
%cd /kaggle/input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master
from efficientnet_pytorch import EfficientNet
%cd -

In [ ]:
import pandas as pd
import numpy as np
from fastai.vision.all import *
import pickle
import os

In [ ]:
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
        os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/resnet50/resnet50.pth' '/root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth'
!cp '../input/resnet101/resnet101.pth' '/root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth'

!cp '../input/efficientnet-pytorch-pretrained/adv-efficientnet-b7-4652b6dd.pth' '/root/.cache/torch/hub/checkpoints/'

!cp '../input/efficientnet-pytorch-pretrained/adv-efficientnet-b6-ac80338e.pth' '/root/.cache/torch/hub/checkpoints/'

 
!cp '../input/efficientnet-pytorch-pretrained/adv-efficientnet-b5-86493f6b.pth' '/root/.cache/torch/hub/checkpoints/'

!cp '../input/efficientnet-pytorch-pretrained/adv-efficientnet-b4-44fb3a87.pth' '/root/.cache/torch/hub/checkpoints/'



!cp '../input/efficientnet-pytorch/efficientnet-b1-dbc7070a.pth' '/root/.cache/torch/hub/checkpoints/'
!cp '../input/efficientnet-pytorch/efficientnet-b2-27687264.pth' '/root/.cache/torch/hub/checkpoints/'
!cp '../input/efficientnet-pytorch/efficientnet-b3-c8376fa2.pth' '/root/.cache/torch/hub/checkpoints/'
!cp '../input/efficientnet-pytorch/efficientnet-b4-e116e8b3.pth' '/root/.cache/torch/hub/checkpoints/'
!cp '../input/efficientnet-pytorch/efficientnet-b5-586e6cc6.pth' '/root/.cache/torch/hub/checkpoints/'
!cp '../input/efficientnet-pytorch/efficientnet-b6-c76e70fd.pth' '/root/.cache/torch/hub/checkpoints/'
!cp '../input/efficientnet-pytorch/efficientnet-b7-dcc49843.pth' '/root/.cache/torch/hub/checkpoints/'

!cp '../input/vgg16weight/vgg16_bn-6c64b313.pth' '/root/.cache/torch/hub/checkpoints/'

In [ ]:
path = Path('../input/hpa-cell-tiles-sample-balanced-dataset')

In [ ]:
df = pd.read_csv(path/'cell_df.csv')
df.head()

In [ ]:
labels = [str(i) for i in range(19)]
for x in labels: df[x] = df['image_labels'].apply(lambda r: int(x in r.split('|')))

## Change below to `frac=1` to run on the whole training sample

In [ ]:
dfs = df.sample(frac=1, random_state=42)
dfs = dfs.reset_index(drop=True)
len(dfs)

In [ ]:
unique_counts = {}
for lbl in labels:
    unique_counts[lbl] = len(dfs[dfs.image_labels == lbl])

full_counts = {}
for lbl in labels:
    count = 0
    for row_label in dfs['image_labels']:
        if lbl in row_label.split('|'): count += 1
    full_counts[lbl] = count
    
counts = list(zip(full_counts.keys(), full_counts.values(), unique_counts.values()))
counts = np.array(sorted(counts, key=lambda x:-x[1]))
counts = pd.DataFrame(counts, columns=['label', 'full_count', 'unique_count'])
counts.set_index('label').T

In [ ]:
len(dfs)

In [ ]:
nfold = 5
seed = 42

y = dfs[labels].values
X = dfs[['image_id', 'cell_id']].values

dfs['fold'] = np.nan

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
mskf = MultilabelStratifiedKFold(n_splits=nfold, random_state=seed)
for i, (_, test_index) in enumerate(mskf.split(X, y)):
    dfs.iloc[test_index, -1] = i
    
dfs['fold'] = dfs['fold'].astype('int')

In [ ]:
dfs.shape

In [ ]:
dfs['is_valid'] = False
dfs['is_valid'][dfs['fold'] == 3] = True

In [ ]:
def get_x(r): return path/'cells'/(r['image_id']+'_'+str(r['cell_id'])+'.jpg')
img = get_x(dfs.loc[12])
img = PILImage.create(img)
img.show();

In [ ]:
def get_y(r): return r['image_labels'].split('|')
get_y(dfs.loc[12])

In [ ]:
sample_stats = ([0.07237246, 0.04476176, 0.07661699], [0.17179589, 0.10284516, 0.14199627])

In [ ]:
item_tfms = RandomResizedCrop(224, min_scale=0.75, ratio=(1.,1.))
batch_tfms = [*aug_transforms(flip_vert=True, size=128, max_warp=0), Normalize.from_stats(*sample_stats)]
bs=128

In [ ]:
dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock(vocab=labels)),
                splitter=ColSplitter(col='is_valid'),
                get_x=get_x,
                get_y=get_y,
                item_tfms=item_tfms,
                batch_tfms=batch_tfms
                )
dls = dblock.dataloaders(dfs, bs=bs)

In [ ]:
dls.show_batch(nrows=3, ncols=3)

## Let's train!

In [ ]:
def get_learner(lr=1e-3):
    opt_func = partial(Adam, lr=lr, wd=0.01, eps=1e-8)    
    model = EfficientNet.from_pretrained("efficientnet-b7", advprop=True)
    model._fc = nn.Linear(2560, dls.c)

    learn = Learner(
        dls, model, opt_func=opt_func,
        metrics=[accuracy_multi, PrecisionMulti()]
        ).to_fp16()
    return learn

In [ ]:
learn=get_learner()

In [ ]:
lr=3e-2

In [ ]:
learn.fine_tune(2,base_lr=lr)

In [ ]:
learn.export('effb7_e2.pkl');
export_learner = load_learner('effb7_e2.pkl')

In [ ]:
learn.recorder.plot_loss()